In [2]:
파일ID = 'https://drive.google.com/file/d/1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3/'
파일저장이름 = 'pytorch_model.bin'

!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3/' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3" -O pytorch_model.bin && rm -rf ~/cookies.txt


파일ID = 'https://drive.google.com/file/d/1-A0Iv_6WGjrdWAsAip4h4vfWZpjfMZB4/'
파일저장이름 = 'config.json'

!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-A0Iv_6WGjrdWAsAip4h4vfWZpjfMZB4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-A0Iv_6WGjrdWAsAip4h4vfWZpjfMZB4" -O config.json && rm -rf ~/cookies.txt

--2024-05-20 07:24:55--  https://docs.google.com/uc?export=download&confirm=&id=1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3
Resolving docs.google.com (docs.google.com)... 74.125.69.139, 74.125.69.101, 74.125.69.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.69.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3&export=download [following]
--2024-05-20 07:24:55--  https://drive.usercontent.google.com/download?id=1cQVhjUQY0Ye6f4PUY96bZiUM4pozhDZ3&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 209.85.145.132, 2607:f8b0:4001:c1e::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|209.85.145.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 0 [application/octet-stream]
Saving to: ‘pytorch_model.bin’

pytorch_model.bin       [ <=>                ]       0  --.-KB/s    in 0s 

In [3]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

import sys

from IPython.display import clear_output
clear_output(wait=True)

## 관련 라이브러스 설치
import tensorflow as tf

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

import keras
# from keras.preprocessing.sequence import pad_sequences  # path removed
from keras.utils import pad_sequences

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

clear_output()

print('Python version : ', sys.version)
print('TensorFlow version : ', tf.__version__)
print('Keras version : ', keras.__version__)

Python version :  3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
TensorFlow version :  2.15.0
Keras version :  2.15.0


In [4]:
bert모델 = "bert-base-multilingual-cased"

# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

import sys

from IPython.display import clear_output
clear_output(wait=True)

import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
# from keras.preprocessing.sequence import pad_sequences  # path removed
from keras.utils import pad_sequences
import numpy as np


## 해당 부분에서 에러가 나면, "런타임 > 런타임 유형 변경 > 하드웨어 가속시 > GPU"로 변경해야 합니다.

# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


tokenizer = BertTokenizer.from_pretrained(bert모델, do_lower_case=False)

#model_fn = 'pytorch_model.bin'
#bert_model = 'bert-base-multilingual-cased'
#model_state_dict = torch.load(model_fn)
#model = BertForSequenceClassification.from_pretrained(bert_model, state_dict = model_state_dict, num_labels = 2)
#model.bert.load_state_dict(model.bert.state_dict())

model = BertForSequenceClassification.from_pretrained(".", num_labels=2)
model.cuda()


clear_output()

print('환경 설정이 완료되었습니다.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

OSError: Unable to load weights from pytorch checkpoint file for './pytorch_model.bin' at './pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:
테스트용텍스트 = '\u5C39\uC774 \uC5F4\uCC3D\uD55C \uB178\uB798, \uBC14\uC774\uB4E0\uC774 \uC8FD\uC740 \uC7A5\uB0A8\uACFC \uC990\uACA8 \uBD88\uB800\uB358 \uACE1\uC774\uC5C8\uB2E4'


# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks


# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


logits = test_sentences([테스트용텍스트])

print(logits)
print(np.argmax(logits))